In [44]:
import warnings

import joblib
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from tqdm import tqdm

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [45]:
path = "./data/preprocessing"

busan_spot_info = pd.read_csv("./data/preprocessing/busan_spot_info - final.csv", encoding="cp949")
travel = pd.read_csv("./data/preprocessing/travel.csv")
traveller_master = pd.read_csv("./data/preprocessing/traveller_master.csv")

In [46]:
def mission_check(travel):
    travel_list = []
    for i in range(len(travel)):
        value = int(travel['TRAVEL_MISSION_CHECK'][i].split(';')[0])
        travel_list.append(value)
    return travel_list


travel['TRAVEL_MISSION_PRIORITY'] = mission_check(travel)
travel = travel[['TRAVEL_ID', 'TRAVELER_ID', 'TRAVEL_MISSION_PRIORITY', 'MVMN_NM']]

In [47]:
# 캠핑 삭제
# 스파/온천/성수 준비 삭제
# 반려동물 동반 여행 삭제
# 친환경 여행 삭제
# 등반 여행 삭제
exclude_priorities = [8, 10, 25, 27, 28]
travel = travel[~travel['TRAVEL_MISSION_PRIORITY'].isin(exclude_priorities)]

In [48]:
travel_style_columns = ['TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 
                        'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8']


traveller_master['TRAVEL_STYL'] = traveller_master[travel_style_columns].mode(axis=1)[0]


traveller_master = traveller_master[['TRAVELER_ID', 'GENDER', 'AGE_GRP', 'INCOME', 'TRAVEL_STYL', 
                                      'TRAVEL_MOTIVE_1', 'TRAVEL_NUM', 'TRAVEL_COMPANIONS_NUM' ]]

In [49]:
busan_spot_info = busan_spot_info[['TRAVEL_ID', 'VISIT_AREA_NM', 'SIDO', 'GUNGU', 'VISIT_AREA_TYPE_CD', 'DGSTFN',
                                  'REVISIT_INTENTION', 'RCMDTN_INTENTION', 'RESIDENCE_TIME_MIN', 'REVISIT_YN']].reset_index(drop=True)

In [50]:
"""Age Group 40대 이상은 40으로 변경"""
traveller_master['AGE_GRP'] = traveller_master['AGE_GRP'].apply(lambda x: 40 if x >= 40 else x)

In [51]:
"""busan spot info와 traveller_master, travel merge Result: 부산에 여행온 사람들의 정보"""
df = pd.merge(travel, traveller_master, left_on = 'TRAVELER_ID', right_on = 'TRAVELER_ID', how = 'inner')
df = pd.merge(busan_spot_info, df, left_on = 'TRAVEL_ID', right_on = 'TRAVEL_ID', how = 'left')  

In [52]:
"""Preprocessing"""

df["RESIDENCE_TIME_MIN"] = df["RESIDENCE_TIME_MIN"].replace(0, df["RESIDENCE_TIME_MIN"].median())

df['REVISIT_YN'] = df['REVISIT_YN'].replace("N",0)
df['REVISIT_YN'] = df['REVISIT_YN'].replace("Y",1)

df = df.dropna(subset = ['TRAVEL_STYL'])
df = df.reset_index(drop= True)
df = df.dropna(subset = ['TRAVEL_MOTIVE_1'])
df = df.reset_index(drop= True)

In [53]:
"""특정 관광지에 방문한 사람들 중 랜덤으로 한 명을 선택하여 그 사람의 모든 여행 데이터를 새로운 데이터프레임 train_df에 추가합니다"""
df_copy = df.copy()
train_df = pd.DataFrame(columns=df.columns)  

for area in tqdm(df['VISIT_AREA_NM'].unique()):
    area_visitors_df = df_copy[df_copy['VISIT_AREA_NM'] == area]  # 특정 관광지에 방문한 모든 사람의 데이터를 추출
    
    np.random.seed(42)  # 랜덤 시드 설정
    if area_visitors_df.empty:  # 만약 해당 관광지에 방문한 사람이 없으면
        continue  # 아무 작업도 하지 않고 다음 반복으로 넘어감
    
    random_idx = np.random.randint(len(area_visitors_df))  # 해당 관광지에 방문한 사람 중 랜덤으로 한 명의 인덱스를 생성
    random_visitor_df = area_visitors_df.iloc[[random_idx]]  # 랜덤으로 선택된 사람의 데이터 추출
    travel_id = random_visitor_df.iloc[0, 0]  # 해당 사람의 'TRAVEL_ID' 값을 추출
    
    visitor_trips_df = df_copy[df_copy['TRAVEL_ID'] == travel_id]  # 선택된 사람의 모든 여행 데이터를 추출
    
    df_copy = pd.merge(visitor_trips_df, df_copy, how='outer', indicator=True)  # 원본 데이터프레임과 선택된 사람의 데이터를 병합, 병합 결과를 표시
    df_copy = df_copy.query('_merge == "right_only"').drop(columns=['_merge'])  # 원본 데이터프레임에서 선택된 사람의 데이터를 삭제
    
    train_df = train_df.append(visitor_trips_df, ignore_index=True)  # 선택된 사람의 데이터를 train_df 데이터프레임에 추가

100%|██████████| 649/649 [00:05<00:00, 129.41it/s]


In [54]:
while len(df_copy) / len(df) > 0.15:
    np.random.seed(42)  # 랜덤 시드 설정
    random_idx = np.random.randint(len(df_copy))  # 랜덤 인덱스 생성
    random_visitor = df_copy.iloc[[random_idx]]  # 랜덤으로 선택된 사람의 데이터 추출
    travel_id = random_visitor.iloc[0, 0]  # 해당 사람의 'TRAVEL_ID' 값 추출
    
    visitor_trips = df_copy[df_copy['TRAVEL_ID'] == travel_id]  # 선택된 사람의 모든 여행 데이터 추출

    df_copy = pd.merge(visitor_trips, df_copy, how='outer', indicator=True)  # 원본 데이터프레임과 선택된 사람의 데이터를 병합, 병합 결과를 표시
    df_copy = df_copy.query('_merge == "right_only"').drop(columns=['_merge'])  # 원본 데이터프레임에서 선택된 사람의 데이터를 삭제
    
    train_df = train_df.append(visitor_trips, ignore_index=True)  # 선택된 사람의 데이터를 train_df 데이터프레임에 추가

In [55]:
new_train = pd.DataFrame(columns = list(train_df.columns) + ['RESIDENCE_TIME_MIN_mean', 'RCMDTN_INTENTION_mean',
                                                          'REVISIT_YN_mean', 'TRAVEL_COMPANIONS_NUM_mean',
                                                          'REVISIT_INTENTION_mean'])


for i in tqdm(list(train_df['VISIT_AREA_NM'].unique())): #유니크한 관광지 목록 중에서
    df2 = train_df[train_df['VISIT_AREA_NM'] == i] # 특정 관광지에 간 모든 사람 뽑아서
    for j in ['RESIDENCE_TIME_MIN', 'RCMDTN_INTENTION', 'REVISIT_YN', 'TRAVEL_COMPANIONS_NUM', 'REVISIT_INTENTION']:
        #체류시간 평균 산출 
        globals()[str(j)+'_mean'] = df2[str(j)]
        globals()[str(j)+'_mean'] = np.mean(globals()[str(j)+'_mean'])
        #데이터프레임에 들어가게 값을 리스트 형태로 변환
        globals()[str(j)+'_mean'] = np.repeat(globals()[str(j)+'_mean'], len(df2)) 
        df2[str(j)+'_mean'] = globals()[str(j)+'_mean']
    #새로운 데이터프레임에 방문지별 평균값 대입
    new_train = pd.concat([new_train, df2], axis = 0)
    
new_train.sort_values(by = ['TRAVEL_ID'], axis = 0, inplace = True)

100%|██████████| 649/649 [00:00<00:00, 709.70it/s]


In [56]:
path = r"C:\workspace\Ko-Swipe-ML\data\preprocessing"
new_train.to_csv(path + '/관광지 추천시스템 Trainset_B.csv', index = False)
df_copy.to_csv(path + '/관광지 추천시스템 Testset_B.csv', index = False)

In [57]:
Train = pd.read_csv(path + '/관광지 추천시스템 Trainset_B.csv')
test = pd.read_csv(path + '/관광지 추천시스템 Testset_B.csv')

Train.dropna(inplace = True)
Train.reset_index(inplace = True, drop = True )

test.dropna(inplace = True)
test.reset_index(inplace = True, drop = True )

print(Train.shape)
print(test.shape)

(2491, 25)
(427, 20)


In [58]:
"""5번 이상 방문한 곳으로만 필터링"""
Train.reset_index(drop = True, inplace= True)

count = pd.DataFrame(Train['VISIT_AREA_NM'].value_counts())
count['places'] = count.index
five_places = list(count[count['VISIT_AREA_NM']>=5]['places']) 
for i in range(len(Train)):
    if Train['VISIT_AREA_NM'][i] not in five_places:
        Train = Train.drop([i], axis = 0)
Train.reset_index(drop = True, inplace = True)

In [59]:
count_visit5 = pd.DataFrame(Train['VISIT_AREA_NM'].value_counts())

In [60]:
"""Drop Features"""
Train.drop(['TRAVELER_ID', 'REVISIT_INTENTION', 'RCMDTN_INTENTION','RESIDENCE_TIME_MIN', 'REVISIT_YN'], axis = 1, inplace = True)
test.drop(['TRAVELER_ID', 'REVISIT_INTENTION', 'RCMDTN_INTENTION','RESIDENCE_TIME_MIN', 'REVISIT_YN'], axis = 1, inplace = True)

In [61]:
"""Change dtype"""
Train['VISIT_AREA_TYPE_CD'] = Train['VISIT_AREA_TYPE_CD'].astype('string')
test['VISIT_AREA_TYPE_CD'] = test['VISIT_AREA_TYPE_CD'].astype('string')

In [62]:
x_train = Train.drop(['DGSTFN', 'TRAVEL_ID'], axis = 1)
y_train = Train['DGSTFN']

In [63]:
category_features = ["VISIT_AREA_NM", "SIDO", "GUNGU", "VISIT_AREA_TYPE_CD", "TRAVEL_MISSION_PRIORITY", "AGE_GRP", "GENDER", "MVMN_NM"]

In [64]:
x_train['TRAVEL_MISSION_PRIORITY'] = x_train['TRAVEL_MISSION_PRIORITY'].astype('int')
x_train['AGE_GRP'] = x_train['AGE_GRP'].astype('int')

In [65]:
model = CatBoostRegressor(cat_features=category_features, random_state=42, task_type="GPU", verbose=100)
model.fit(x_train, y_train)

Learning rate set to 0.043665
0:	learn: 0.8046557	total: 15.3ms	remaining: 15.3s
100:	learn: 0.7137837	total: 1.73s	remaining: 15.4s
200:	learn: 0.6855239	total: 3.45s	remaining: 13.7s
300:	learn: 0.6677048	total: 5.12s	remaining: 11.9s
400:	learn: 0.6501355	total: 6.75s	remaining: 10.1s
500:	learn: 0.6336369	total: 8.43s	remaining: 8.39s
600:	learn: 0.6186181	total: 10.1s	remaining: 6.71s
700:	learn: 0.6065598	total: 11.7s	remaining: 4.99s
800:	learn: 0.5950835	total: 13.3s	remaining: 3.29s
900:	learn: 0.5846477	total: 14.8s	remaining: 1.63s
999:	learn: 0.5775134	total: 16.4s	remaining: 0us


In [66]:
joblib.dump(model, path + '/catboost_model_B.pkl')

['C:\\workspace\\Ko-Swipe-ML\\data\\preprocessing/catboost_model_B.pkl']

In [83]:
path = r"C:\workspace\Ko-Swipe-ML\data\preprocessing"

modelb = joblib.load(path + '/catboost_model_B.pkl')
testb = pd.read_csv(path + '/관광지 추천시스템 Testset_B.csv')
trainb = pd.read_csv(path + '/관광지 추천시스템 Trainset_B.csv')

trainb.dropna(inplace = True)
trainb.reset_index(inplace = True, drop = True )

testb.dropna(inplace = True)
testb.reset_index(inplace = True, drop = True )


In [84]:
X_testb = testb.drop(['DGSTFN'], axis = 1)
y_testb = testb['DGSTFN']

In [85]:
# 유저정보
user_info = testb[
    [
        "TRAVEL_ID",
        "SIDO",
        "GUNGU",
        "TRAVEL_MISSION_PRIORITY",
        "MVMN_NM",
        "GENDER",
        "AGE_GRP",
        "INCOME",
        "TRAVEL_STYL",
        "TRAVEL_MOTIVE_1",
        "TRAVEL_NUM",
        "TRAVEL_COMPANIONS_NUM",
    ]
]

In [86]:
new_user_info = pd.DataFrame(columns=['TRAVEL_ID', 'TRAVEL_MISSION_PRIORITY', "MVMN_NM", 'GENDER', 'AGE_GRP', 'INCOME',
                            'TRAVEL_STYL','TRAVEL_MOTIVE_1', 'TRAVEL_NUM', 'TRAVEL_COMPANIONS_NUM', 'sido_gungu_list'])

for i in tqdm(list(user_info['TRAVEL_ID'].unique())):
    user_info_filtered = user_info[user_info['TRAVEL_ID'] == i]
    userLocations = user_info_filtered[['SIDO', 'GUNGU']] #각 유저별 방문한 시군구 확인
    userLocations.reset_index(drop = True, inplace = True)
    sido_gungu_visit = []
    for j in range(len(userLocations)):
        sido_gungu_visit.append(userLocations['SIDO'][j] + '+' + userLocations['GUNGU'][j])
    sido_gungu_list = list(set(sido_gungu_visit))
    new = user_info_filtered.drop(['SIDO', 'GUNGU'], axis = 1) #기존 시도, 군구 제외하고
    new = new.head(1)
    new['sido_gungu_list'] = str(sido_gungu_list)
    new_user_info = pd.concat([new_user_info, new], axis = 0) #새로운 데이터프레임 생성        
    

100%|██████████| 187/187 [00:00<00:00, 1015.72it/s]


In [87]:
new_user_info.reset_index(drop = True, inplace = True)
new_user_info.to_csv(path + '/관광지 추천시스템 Testset_B- 유저 정보.csv', index=False)

In [88]:
#여행지 정보
info = trainb[['SIDO', 'VISIT_AREA_NM', 'GUNGU','VISIT_AREA_TYPE_CD','RESIDENCE_TIME_MIN_mean', 'RCMDTN_INTENTION_mean',
            'REVISIT_YN_mean', 'TRAVEL_COMPANIONS_NUM_mean', 'REVISIT_INTENTION_mean']]
info.drop_duplicates(['VISIT_AREA_NM'], inplace = True)

In [89]:
###### 2회 이상 관광한 방문지 리스트 생성
visiting_list = trainb[['VISIT_AREA_NM']] #train set에 있는 방문지에 대해서만 2회 이상 방문하였는지 확인
visiting_list.reset_index(drop = True, inplace = True)
#데이터 전처리
visit_count_df = pd.DataFrame(visiting_list.value_counts(), columns = ['count'])
visit_count_df['VISIT_AREA_NM'] = visit_count_df.index
visit_count_df.reset_index(drop = True, inplace = True)
for i in range(len(visit_count_df)):
    visit_count_df['VISIT_AREA_NM'][i] = str(visit_count_df['VISIT_AREA_NM'][i])
    visit_count_df['VISIT_AREA_NM'][i] = visit_count_df['VISIT_AREA_NM'][i].replace("(","").replace(")","").replace(",","").replace("\''","")
    visit_count_df['VISIT_AREA_NM'][i] = visit_count_df['VISIT_AREA_NM'][i][1:-1]
#5회 이상 적용
visit_count_df = visit_count_df[visit_count_df['count'] >= 5]  
visit_list = list(visit_count_df['VISIT_AREA_NM']) #visit_list에 5회 이상 방문지 리스트

In [90]:
#방문지가 5회 이상 방문한 관광지 아니면 제거
info.reset_index(drop = True, inplace = True)
for i in tqdm(range(len(info))):
    if info['VISIT_AREA_NM'][i] not in visit_list:
        info = info.drop([i], axis = 0)
info.reset_index(drop = True, inplace = True)

100%|██████████| 602/602 [00:00<00:00, 8123.92it/s]


In [91]:
#여행자 정보 저장
info.reset_index(drop = True, inplace = True)
info.to_csv(path + '/관광지 추천시스템 Testset_B- 여행지 정보.csv', index=False)

In [100]:
info

,SIDO,VISIT_AREA_NM,GUNGU,VISIT_AREA_TYPE_CD,RESIDENCE_TIME_MIN_mean,RCMDTN_INTENTION_mean,REVISIT_YN_mean,TRAVEL_COMPANIONS_NUM_mean,REVISIT_INTENTION_mean
0,부산,용두산공원,중구,2,43.750000,3.833333,0.583333,0.875000,3.708333
1,부산,부평깡통시장,중구,4,56.718750,4.093750,0.500000,1.156250,4.109375
2,부산,부산 시립미술관,해운대구,3,53.076923,4.307692,0.461538,1.076923,4.384615
3,부산,송정해수욕장,해운대구,1,73.043478,4.500000,0.326087,1.673913,4.521739
4,부산,죽성드림세트장,기장군,2,40.714286,4.357143,0.000000,1.642857,4.000000
...,...,...,...,...,...,...,...,...,...
73,부산,행리단길,해운대구,4,55.263158,4.473684,0.315789,0.894737,4.315789
74,부산,롯데 프리미엄 아웃렛 동부산점,기장군,4,96.666667,4.611111,0.777778,1.388889,4.666667
75,부산,아홉 살 숲,기장군,1,93.750000,4.125000,0.125000,1.875000,4.250000
76,부산,서면 미술관,부산진구,3,50.000000,4.166667,0.000000,0.666667,3.500000


In [92]:
user_info = pd.read_csv(path + '/관광지 추천시스템 Testset_B- 유저 정보.csv')
info = pd.read_csv(path + '/관광지 추천시스템 Testset_B- 여행지 정보.csv')

In [93]:
new_user_info['TRAVEL_MISSION_PRIORITY'] = new_user_info['TRAVEL_MISSION_PRIORITY'].astype('int')
new_user_info['AGE_GRP'] = new_user_info['AGE_GRP'].astype('int')

In [94]:
new_user_info

,TRAVEL_ID,TRAVEL_MISSION_PRIORITY,MVMN_NM,GENDER,AGE_GRP,INCOME,TRAVEL_STYL,TRAVEL_MOTIVE_1,TRAVEL_NUM,TRAVEL_COMPANIONS_NUM,sido_gungu_list
0,b_b006879,2,자가용,여,30,2.0,4.0,7.0,3.0,1.0,['부산+사하구']
1,b_b004725,1,자가용,남,30,6.0,2.0,3.0,2.0,1.0,['부산+기장군']
2,b_b004714,1,자가용,여,40,1.0,2.0,1.0,10.0,1.0,['부산+중구']
3,b_b014149,2,자가용,여,20,1.0,1.0,3.0,1.0,1.0,['부산+기장군']
4,b_b007627,6,대중교통 등,여,30,5.0,3.0,7.0,1.0,3.0,['부산+사하구']
...,...,...,...,...,...,...,...,...,...,...,...
182,b_b012328,23,대중교통 등,남,20,4.0,6.0,4.0,3.0,1.0,['부산+수영구']
183,b_b006514,22,자가용,여,20,4.0,2.0,1.0,2.0,0.0,['부산+중구']
184,d_d005737,21,자가용,여,40,4.0,4.0,2.0,7.0,0.0,['부산+수영구']
185,b_b004235,6,자가용,남,30,6.0,4.0,7.0,3.0,0.0,['부산+수영구']


In [95]:
test_data = pd.DataFrame(user_info.iloc[1]).T

test_data.drop(['TRAVEL_ID'], axis = 1, inplace = True)

test_data

,TRAVEL_MISSION_PRIORITY,MVMN_NM,GENDER,AGE_GRP,INCOME,TRAVEL_STYL,TRAVEL_MOTIVE_1,TRAVEL_NUM,TRAVEL_COMPANIONS_NUM,sido_gungu_list
1,1.0,자가용,남,30.0,6.0,2.0,3.0,2.0,1.0,['부산+기장군']


In [96]:
import pandas as pd

# Define column lists
final_columns = [
    "VISIT_AREA_NM", "SIDO", "GUNGU", "VISIT_AREA_TYPE_CD", "TRAVEL_MISSION_PRIORITY", "MVMN_NM", "GENDER", "AGE_GRP", "INCOME",
    "TRAVEL_STYL", "TRAVEL_MOTIVE_1", "TRAVEL_NUM", "TRAVEL_COMPANIONS_NUM", 
    "RESIDENCE_TIME_MIN_mean", "RCMDTN_INTENTION_mean", "REVISIT_YN_mean", "TRAVEL_COMPANIONS_NUM_mean", 
    "REVISIT_INTENTION_mean"
]

user_columns = [
    "SIDO", "TRAVEL_MISSION_PRIORITY", "MVMN_NM", "GENDER", "AGE_GRP", "INCOME", "TRAVEL_STYL",
    "TRAVEL_MOTIVE_1", "TRAVEL_NUM", "TRAVEL_COMPANIONS_NUM"
]

features = final_columns

def convert_float_to_int(df):
    float_cols = df.select_dtypes(include=['float']).columns
    
    for col in float_cols:
        df[col] = df[col].astype(int)
    
    return df

def preprocess_places_list(places_list_str):
    """
    Preprocesses the places list string into a list of places.

    Parameters:
    places_list_str (str): String representation of the places list.

    Returns:
    list: A list of places.
    """
    places_list = places_list_str.replace("[", "").replace("]", "").replace("'", "").replace(", ", ",")
    return list(map(str, places_list.split(",")))

def generate_final_df(info, new_user_info, places_list):
    """
    Generates the final DataFrame based on user information and places list.

    Parameters:
    info (DataFrame): DataFrame containing area information.
    new_user_info (DataFrame): DataFrame containing new user information.
    places_list (list): List of places.

    Returns:
    DataFrame: The final DataFrame containing combined user and area information.
    """
    final_df = pd.DataFrame(columns=final_columns)
    
    for place in places_list:
        sido, gungu = map(str, place.split("+"))
        info_df = info[(info["SIDO"] == sido) & (info["GUNGU"] == gungu)].drop(["SIDO"], axis=1).reset_index(drop=True)
        user_data = new_user_info.drop(["sido_gungu_list"], axis=1).values.tolist()[0]
        user_data = [sido] + user_data
        user_df = pd.DataFrame([user_data] * len(info_df), columns=user_columns)
        df = pd.concat([user_df, info_df], axis=1)[features]
        df["VISIT_AREA_TYPE_CD"] = df["VISIT_AREA_TYPE_CD"].astype("string")
        final_df = pd.concat([final_df, df], axis=0)
        
    final_df.reset_index(drop=True, inplace=True)
    final_df.drop_duplicates(["VISIT_AREA_NM"], inplace=True)
    return final_df

def recommend_places(model, final_df):
    """
    Recommends places based on the model's predictions.

    Parameters:
    model: The predictive model.
    final_df (DataFrame): The final DataFrame containing combined user and area information.

    Returns:
    list: List of recommended places.
    """
    final_df = convert_float_to_int(final_df)
    y_pred = model.predict(final_df)
    y_pred_df = pd.DataFrame(y_pred, columns=["y_pred"])
    sorted_df = pd.concat([final_df, y_pred_df], axis=1).sort_values(by="y_pred", ascending=False).iloc[:10]
    return list(sorted_df["VISIT_AREA_NM"])


def generate_user_info_df(final_df):
    """
    Generates a DataFrame containing user information.

    Parameters:
    final_df (DataFrame): The final DataFrame containing combined user and area information.

    Returns:
    DataFrame: DataFrame containing user information.
    """
    return final_df[user_columns]

def main(info, new_user_info, model):
    """
    Main function to generate recommendations and user information.

    Parameters:
    info (DataFrame): DataFrame containing area information.
    new_user_info (DataFrame): DataFrame containing new user information.
    model: The predictive model.

    Returns:
    list: A list containing user information and recommended places.
    """
    
    
    result = []
    places_list_str = new_user_info["sido_gungu_list"].values[0]
    places_list = preprocess_places_list(places_list_str)
    final_df = generate_final_df(info, new_user_info, places_list)
    
    visiting_candidates = recommend_places(model, final_df)
    user_info_df = generate_user_info_df(final_df)
    
    if len(user_info_df) == 0:
        result.append([])
    else:
        rec = user_info_df.iloc[0].to_list()
        rec.append(visiting_candidates)
        result.append(rec)
    
    return result

#Example usage:
result = main(info, test_data, modelb)
print(result)


[['부산', 1, '자가용', '남', 30, 6, 2, 3, 2, 1, ['죽성드림세트장', '스카이라인루지 부산', '롯데월드 어드벤처 부산', '해동용궁사', '아홉 살 숲', '이케아 동부산점', '롯데프리미엄아울렛 동부산점', '롯데 프리미엄 아웃렛 동부산점']]]


In [105]:
result[0][-1]

['죽성드림세트장',
 '스카이라인루지 부산',
 '롯데월드 어드벤처 부산',
 '해동용궁사',
 '아홉 살 숲',
 '이케아 동부산점',
 '롯데프리미엄아울렛 동부산점',
 '롯데 프리미엄 아웃렛 동부산점']